# NI vs A-FVS

The following are preliminary experiments to compare the results of the Alpha Fast Vertex Substitution (A-FVS) performance against a straightforward implementation of the interchange local search heuristic, called Naive Interchange (NI).
The initial solution, either RGD or random, is not measured.

In [1]:
from models.instance import Instance
from models.solver import generate_solvers

import os

def get_solvers(n: int, m: int, amount: int):
    datapath = os.path.abspath("..\\data")
    instances = [Instance.read_json(os.path.join(datapath, f"anpcp_{n}_{m}_{i}.json")) for i in range(amount)]
    return generate_solvers(instances, (0.1, 0.25, 0.5), (2, 3))

In [2]:
from utils import compare_local_search

import pandas as pd

def get_results(solvers, n: int, m: int, from_random: bool):
    """
    Compares local search methods.
    Reads a pickle file of the results if it exists,
    otherwise it runs the experiment and writes a new file.
    """
    initial = "rand" if from_random else "rgd"
    results_if = results_ib = None

    # read files, either both exist or don't
    dirpath = ".\\nb_results\\ni_afvs"
    filepath_if = os.path.join(dirpath, f"{initial}_if_{n}_{m}.pkl")
    filepath_ib = os.path.join(dirpath, f"{initial}_ib_{n}_{m}.pkl")

    if os.path.exists(filepath_if):
        results_if = pd.read_pickle(filepath_if)

    if os.path.exists(filepath_ib):
        results_ib = pd.read_pickle(filepath_ib)
    
    if results_if is None or results_ib is None:
        results_if, results_ib = compare_local_search(solvers, from_random)

        results_if.to_pickle(filepath_if)
        results_ib.to_pickle(filepath_ib)

    return results_if, results_ib

In [3]:
from IPython.display import display

def display_results(dataframe: pd.DataFrame, caption: str):
    displayed = dataframe.style.format(precision=2).set_caption(caption)
    display(displayed)

## RGD as initial solution

The constructive heuristic Randomized Greedy Dispersion (RGD) is run first to get an initial solution.

In [4]:
solvers50 = get_solvers(50, 50, 5)
solvers100 = get_solvers(100, 100, 4)
solvers500 = get_solvers(500, 500, 3)

In [5]:
rgd_if_50, rgd_ib_50 = get_results(solvers50, 50, 50, False)
display_results(rgd_if_50, "IF")
display_results(rgd_ib_50, "IB")

In [6]:
rgd_if_100, rgd_ib_100 = get_results(solvers100, 100, 100, False)
display_results(rgd_if_100, "IF")
display_results(rgd_ib_100, "IB")

In [7]:
rgd_if_500, rgd_ib_500 = get_results(solvers500, 500, 500, False)
display_results(rgd_if_500, "IF")
display_results(rgd_ib_500, "IB")

In [8]:
rgd_if = pd.concat([rgd_if_50, rgd_if_100, rgd_if_500])
rgd_ib = pd.concat([rgd_ib_50, rgd_ib_100, rgd_ib_500])
display_results(rgd_if, "IF")
display_results(rgd_ib, "IB")

## Random initial solution

A random solution is now going to be used as the initial solution.

In [9]:
solvers50 = get_solvers(50, 50, 5)
solvers100 = get_solvers(100, 100, 4)
solvers500 = get_solvers(500, 500, 3)

In [10]:
rand_if_50, rand_ib_50 = get_results(solvers50, 50, 50, True)
display_results(rand_if_50, "IF")
display_results(rand_ib_50, "IB")

In [11]:
rand_if_100, rand_ib_100 = get_results(solvers100, 100, 100, True)
display_results(rand_if_100, "IF")
display_results(rand_ib_100, "IB")

In [12]:
rand_if_500, rand_ib_500 = get_results(solvers500, 500, 500, True)
display_results(rand_if_500, "IF")
display_results(rand_ib_500, "IB")

In [13]:
rand_if = pd.concat([rand_if_50, rand_if_100, rand_if_500])
rand_ib = pd.concat([rand_ib_50, rand_ib_100, rand_ib_500])
display_results(rand_if, "IF")
display_results(rand_ib, "IB")

## Time comparison

**How faster is FVS compared to NI?**

In [14]:
def get_timediff(dataframe: pd.DataFrame) -> float:
    timediff = dataframe["NI", "time"] / dataframe["A-FVS", "time"]
    return timediff.mean()

$n = 50$

In [15]:
get_timediff(rgd_if_50)

19.60242812871962

In [16]:
get_timediff(rgd_ib_50)

56.3088318263197

$n = 100$

In [17]:
get_timediff(rgd_if_100)

40.59712961129083

In [18]:
get_timediff(rgd_ib_100)

133.3168847002688

$n = 500$

In [19]:
get_timediff(rgd_if_500)

199.40110268967473

In [20]:
get_timediff(rgd_ib_500)

781.9867431618662

In average for this experiment

In [21]:
get_timediff(rgd_if)

86.53355347656174

In [22]:
get_timediff(rgd_ib)

323.87081989615154

In [23]:
from utils import format_latex_table

dirpath = "nb_results\\ni_afvs"
format_latex_table(rgd_if, f"{dirpath}\\rgd_if.tex")
format_latex_table(rgd_ib, f"{dirpath}\\rgd_ib.tex")
format_latex_table(rand_if, f"{dirpath}\\rand_if.tex")
format_latex_table(rand_ib, f"{dirpath}\\rand_ib.tex")

c:\Users\netoo\local-personal\repos\alpha-neighbor-p-center-problem\anpcp\utils.py:103: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  dataframe.to_latex(path, float_format="%.2f", multirow=True)
